<a href="https://colab.research.google.com/github/nandanajayachandran/NANDANA_J_ICT_ASSIGNMENTS_CASE_STUDIES/blob/main/ASSIGNMENT_6_NANDANA_J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ASSIGNMENT NO: 6 - Machine Learning to predict public sentiment from text data**





In [60]:
import pandas as pd
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**LOAD THE DATASET**

In [61]:
df = pd.read_csv('/content/judge-1377884607_tweet_product_company (1).csv')
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


**PREPROCESSING THE TEXT**

In [62]:
df.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

In [63]:
#Dropping rows with missing values
data = df.dropna()

**DATA CLEANING**

In [64]:
data['tweet_text'] = data['tweet_text'].str.lower()
data['tweet_text'] = data['tweet_text'].replace('[^a-zA-Z\s]', '', regex=True)

<ipython-input-64-1512919e3e7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet_text'] = data['tweet_text'].str.lower()
<ipython-input-64-1512919e3e7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet_text'] = data['tweet_text'].replace('[^a-zA-Z\s]', '', regex=True)


In [65]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['emotion_in_tweet_is_directed_at'] = label_encoder.fit_transform(data['emotion_in_tweet_is_directed_at'])


<ipython-input-65-5cd52b196bf3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['emotion_in_tweet_is_directed_at'] = label_encoder.fit_transform(data['emotion_in_tweet_is_directed_at'])


In [66]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [67]:
#Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['tweet_text'])
sequences = tokenizer.texts_to_sequences(data['tweet_text'])

**TRAIN-TEST SPLIT**

In [68]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data['tweet_text'],
    data['is_there_an_emotion_directed_at_a_brand_or_product'],
    test_size=0.2,
    random_state=42
)

#Initialize the CountVectorizer
vectorizer = CountVectorizer()

#Fitting the training data
X_train= vectorizer.fit_transform(X_train['tweet_text'])

#Transforming the testing data
X_test= vectorizer.transform(X_test['tweet_text'])



In [74]:
#Converting emotion labels to categorical format
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.transform(y_test))

**MODEL CREATION**

In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Masking

In [77]:
#model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=None))
model.add(Masking(mask_value=0.0))
model.add(LSTM(units=100))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

#Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [81]:

 import numpy as np
X_train_array = X_train.toarray()
#Train the model
model.fit(X_train_array, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
74/74 [==============================] - 607s 8s/step - loss: 0.6727 - accuracy: 0.8019 - val_loss: 0.5780 - val_accuracy: 0.8220
Epoch 2/5
74/74 [==============================] - 586s 8s/step - loss: 0.5824 - accuracy: 0.8108 - val_loss: 0.5739 - val_accuracy: 0.8220
Epoch 3/5
74/74 [==============================] - 600s 8s/step - loss: 0.5810 - accuracy: 0.8108 - val_loss: 0.5851 - val_accuracy: 0.8220
Epoch 4/5
74/74 [==============================] - 584s 8s/step - loss: 0.5838 - accuracy: 0.8108 - val_loss: 0.5788 - val_accuracy: 0.8220
Epoch 5/5
74/74 [==============================] - 576s 8s/step - loss: 0.5811 - accuracy: 0.8108 - val_loss: 0.5754 - val_accuracy: 0.8220


**PREDICTION**

In [86]:
#Example new text data
new_text = ["I love Apple products!", "Google's services are good."]

#Vectorize
new_text_features = vectorizer.transform(new_text)

#Make predictions
predictions = model.predict(new_text_features.toarray())

#Predictions to human-readable labels
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

#Printing the predictions
for text, label in zip(new_text, predicted_labels):
    print(f"Text: {text}\nPredicted Sentiment: {label}\n")


1/1 [==============================] - 1s 984ms/step
Text: I love Apple products!
Predicted Sentiment: Positive emotion

Text: Google's services are good.
Predicted Sentiment: Positive emotion

